<a href="https://colab.research.google.com/github/MnCSSJ4x/Content-based-recommendation/blob/main/TextToNetflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Netflix Movies Reccomendation System 
Goal is to suggest a movie if a user liked a specific movie from the list of movies on Netflix 

In [31]:
import numpy as np 
import pandas as pd 
import re
import tqdm 
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics.pairwise import cosine_similarity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
df = pd.read_csv('Top_10000_Movies.csv',lineterminator='\n')

In [33]:
df.head()

,Unnamed: 0,id,original_language,original_title,popularity,release_date,vote_average,vote_count,genre,overview,revenue,runtime,tagline
0,0,580489,en,Venom: Let There Be Carnage,5401.308,2021-09-30,6.8,1736,"['Science Fiction', 'Action', 'Adventure']",After finding a host body in investigative rep...,424000000,97.0,NaN
1,1,524434,en,Eternals,3365.535,2021-11-03,7.1,622,"['Action', 'Adventure', 'Science Fiction', 'Fa...",The Eternals are a team of ancient aliens who ...,165000000,157.0,In the beginning...
2,2,438631,en,Dune,2911.423,2021-09-15,8.0,3632,"['Action', 'Adventure', 'Science Fiction']","Paul Atreides, a brilliant and gifted young ma...",331116356,155.0,"Beyond fear, destiny awaits."
3,3,796499,en,Army of Thieves,2552.437,2021-10-27,6.9,555,"['Action', 'Crime', 'Thriller']",A mysterious woman recruits bank teller Ludwig...,0,127.0,"Before Vegas, one locksmith became a legend."
4,4,550988,en,Free Guy,1850.470,2021-08-11,7.8,3493,"['Comedy', 'Action', 'Adventure', 'Science Fic...",A bank teller called Guy realizes he is a back...,331096766,115.0,Life's too short to be a background character.


In [34]:
df.drop(labels=['Unnamed: 0'],axis="columns",inplace=True)

In [35]:
df = df[df['original_language']=='en']

In [36]:
df.shape

(7798, 12)

In [37]:
df = df.dropna(subset=['original_title','overview','genre'])

In [38]:
df = df.drop(df[df['original_title'] == ''].index)
df = df.drop(df[df['overview'] == ''].index)
df = df.drop(df[df['genre'] == ''].index)
df

,id,original_language,original_title,popularity,release_date,vote_average,vote_count,genre,overview,revenue,runtime,tagline
0,580489,en,Venom: Let There Be Carnage,5401.308,2021-09-30,6.8,1736,"['Science Fiction', 'Action', 'Adventure']",After finding a host body in investigative rep...,424000000,97.0,NaN
1,524434,en,Eternals,3365.535,2021-11-03,7.1,622,"['Action', 'Adventure', 'Science Fiction', 'Fa...",The Eternals are a team of ancient aliens who ...,165000000,157.0,In the beginning...
2,438631,en,Dune,2911.423,2021-09-15,8.0,3632,"['Action', 'Adventure', 'Science Fiction']","Paul Atreides, a brilliant and gifted young ma...",331116356,155.0,"Beyond fear, destiny awaits."
3,796499,en,Army of Thieves,2552.437,2021-10-27,6.9,555,"['Action', 'Crime', 'Thriller']",A mysterious woman recruits bank teller Ludwig...,0,127.0,"Before Vegas, one locksmith became a legend."
4,550988,en,Free Guy,1850.470,2021-08-11,7.8,3493,"['Comedy', 'Action', 'Adventure', 'Science Fic...",A bank teller called Guy realizes he is a back...,331096766,115.0,Life's too short to be a background character.
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,530,en,A Grand Day Out,9.266,1990-05-18,7.5,594,"['Adventure', 'Animation', 'Comedy', 'Science ...",Wallace and Gromit have run out of cheese and ...,0,23.0,NaN
9996,15934,en,El cantante,10.417,2006-09-12,7.0,80,"['History', 'Drama', 'Music']","The rise and fall of salsa singer, Héctor Lavo...",0,116.0,"Based on the true story of the King of Salsa, ..."
9997,162215,en,How I Live Now,9.520,2013-09-10,6.6,705,"['Drama', 'Action', 'Thriller', 'War']",An American girl on holiday in the English cou...,0,101.0,Love will lead you home
9998,5723,en,Once,9.267,2007-03-23,7.4,990,"['Drama', 'Music', 'Romance']",A vacuum repairman moonlights as a street musi...,20710513,85.0,How often do you find the right person?


In [39]:
df.columns

Index(['id', 'original_language', 'original_title', 'popularity',
       'release_date', 'vote_average', 'vote_count', 'genre', 'overview',
       'revenue', 'runtime', 'tagline'],
      dtype='object')

In [40]:
def preprocess(text):
  text = re.sub("\'", "", text)
  text = re.sub("[^a-zA-Z]"," ",text)
  text = ' '.join(text.split())
  text = text.lower()
  return text

df['preproc_description'] = df['overview'].apply(lambda x: preprocess(x))
df['preproc_genre'] = df['genre'].apply(lambda x: preprocess(x))
df.head()

,id,original_language,original_title,popularity,release_date,vote_average,vote_count,genre,overview,revenue,runtime,tagline,preproc_description,preproc_genre
0,580489,en,Venom: Let There Be Carnage,5401.308,2021-09-30,6.8,1736,"['Science Fiction', 'Action', 'Adventure']",After finding a host body in investigative rep...,424000000,97.0,NaN,after finding a host body in investigative rep...,science fiction action adventure
1,524434,en,Eternals,3365.535,2021-11-03,7.1,622,"['Action', 'Adventure', 'Science Fiction', 'Fa...",The Eternals are a team of ancient aliens who ...,165000000,157.0,In the beginning...,the eternals are a team of ancient aliens who ...,action adventure science fiction fantasy
2,438631,en,Dune,2911.423,2021-09-15,8.0,3632,"['Action', 'Adventure', 'Science Fiction']","Paul Atreides, a brilliant and gifted young ma...",331116356,155.0,"Beyond fear, destiny awaits.",paul atreides a brilliant and gifted young man...,action adventure science fiction
3,796499,en,Army of Thieves,2552.437,2021-10-27,6.9,555,"['Action', 'Crime', 'Thriller']",A mysterious woman recruits bank teller Ludwig...,0,127.0,"Before Vegas, one locksmith became a legend.",a mysterious woman recruits bank teller ludwig...,action crime thriller
4,550988,en,Free Guy,1850.470,2021-08-11,7.8,3493,"['Comedy', 'Action', 'Adventure', 'Science Fic...",A bank teller called Guy realizes he is a back...,331096766,115.0,Life's too short to be a background character.,a bank teller called guy realizes he is a back...,comedy action adventure science fiction


In [41]:
df['text'] = df["preproc_description"] + " " + df["preproc_genre"]
df['text'] = df['text'].apply(lambda x:x.lower())

In [42]:
df.head()

,id,original_language,original_title,popularity,release_date,vote_average,vote_count,genre,overview,revenue,runtime,tagline,preproc_description,preproc_genre,text
0,580489,en,Venom: Let There Be Carnage,5401.308,2021-09-30,6.8,1736,"['Science Fiction', 'Action', 'Adventure']",After finding a host body in investigative rep...,424000000,97.0,NaN,after finding a host body in investigative rep...,science fiction action adventure,after finding a host body in investigative rep...
1,524434,en,Eternals,3365.535,2021-11-03,7.1,622,"['Action', 'Adventure', 'Science Fiction', 'Fa...",The Eternals are a team of ancient aliens who ...,165000000,157.0,In the beginning...,the eternals are a team of ancient aliens who ...,action adventure science fiction fantasy,the eternals are a team of ancient aliens who ...
2,438631,en,Dune,2911.423,2021-09-15,8.0,3632,"['Action', 'Adventure', 'Science Fiction']","Paul Atreides, a brilliant and gifted young ma...",331116356,155.0,"Beyond fear, destiny awaits.",paul atreides a brilliant and gifted young man...,action adventure science fiction,paul atreides a brilliant and gifted young man...
3,796499,en,Army of Thieves,2552.437,2021-10-27,6.9,555,"['Action', 'Crime', 'Thriller']",A mysterious woman recruits bank teller Ludwig...,0,127.0,"Before Vegas, one locksmith became a legend.",a mysterious woman recruits bank teller ludwig...,action crime thriller,a mysterious woman recruits bank teller ludwig...
4,550988,en,Free Guy,1850.470,2021-08-11,7.8,3493,"['Comedy', 'Action', 'Adventure', 'Science Fic...",A bank teller called Guy realizes he is a back...,331096766,115.0,Life's too short to be a background character.,a bank teller called guy realizes he is a back...,comedy action adventure science fiction,a bank teller called guy realizes he is a back...


In [43]:
ls = []

def prompt(summary):
    ls.append(summary)
    for i in (df["text"]):
        ls.append(i)

In [44]:
model = SentenceTransformer('nli-distilroberta-base-v2')

In [45]:
prompt("Romantic Love story")

In [46]:
embedding = model.encode(ls)
embedding.shape

(7777, 768)

In [47]:
sim = cosine_similarity([embedding[0]], embedding[1:]).flatten()
sim = list(enumerate(sim))
sim.sort(key=lambda x: x[1])
sim.reverse()
sim

[(5924, 0.69888633),
 (5845, 0.69569194),
 (7176, 0.6882043),
 (7555, 0.66787934),
 (2292, 0.64555085),
 (7674, 0.64116085),
 (4343, 0.64050627),
 (6172, 0.63068306),
 (4331, 0.6302371),
 (3651, 0.62763596),
 (5874, 0.6266133),
 (3817, 0.6245341),
 (1735, 0.6148355),
 (4573, 0.61038506),
 (2130, 0.61023915),
 (7722, 0.6062056),
 (6050, 0.605235),
 (7066, 0.6045522),
 (1654, 0.60274863),
 (2734, 0.60247767),
 (1052, 0.6019012),
 (6953, 0.5972286),
 (3306, 0.59506774),
 (4911, 0.59345603),
 (4932, 0.5928292),
 (3911, 0.5926345),
 (6262, 0.59076655),
 (6519, 0.5899837),
 (6019, 0.5895341),
 (4618, 0.5885822),
 (6906, 0.584618),
 (2341, 0.5844343),
 (6944, 0.5828313),
 (3558, 0.5825746),
 (3921, 0.5822004),
 (826, 0.5800849),
 (6214, 0.57902473),
 (1597, 0.57840335),
 (2477, 0.57768154),
 (5283, 0.5762329),
 (6841, 0.57459295),
 (5976, 0.5738617),
 (5082, 0.5730134),
 (1018, 0.5702301),
 (4708, 0.569896),
 (309, 0.5687773),
 (850, 0.5684141),
 (172, 0.5682316),
 (6842, 0.56484395),
 (2438,

In [48]:
recommended = sim[0:5]
for i in recommended:
    print(df.iloc[i[0]])

id                                                                316021
original_language                                                     en
original_title                                              Frank & Lola
popularity                                                        11.132
release_date                                                  2016-12-09
vote_average                                                         5.6
vote_count                                                           130
genre                                    ['Drama', 'Mystery', 'Romance']
overview               A psychosexual noir love story, set in Las Veg...
revenue                                                             9188
runtime                                                             88.0
tagline                                  Love obssesion betrayal revenge
preproc_description    a psychosexual noir love story set in las vega...
preproc_genre                                      